In [9]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from utils.metric import *
# import properscoring as ps
from tqdm import tqdm
plt.rcParams['font.family'] = 'Times New Roman'

In [10]:
M = 20
X = torch.load("./data/load_test.pt").unsqueeze(1).repeat(1, M, 1)
X_hat_gan = torch.load("./result/data/load_hat_gan.pt")[:, :M, :]
X_hat_diff_base = torch.load("./result/data/load_hat_diff_base.pt")[:, :M, :]
X_hat_diff_phy = torch.load("./result/data/load_hat_diff_phy.pt")[:, :M, :]
X_hat_vae = torch.load("./result/data/load_hat_vae.pt")[:, :M, :]
X_hat_nf = torch.load("./result/data/load_hat_nf.pt")[:, :M, :]

/tmp/ipykernel_13898/2687782414.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X = torch.load("./data/load_test.pt").unsqueeze(1).repeat(1, M, 1)
/tmp/ipykernel_13898/2

In [11]:
X_max = torch.max(X[:, 0, :], dim=1)
X_min = torch.min(X[:, 0, :], dim=1)

In [14]:
len(X)

2190

In [12]:

low_th = np.percentile(X_min.values, 1)
high_th = np.percentile(X_max.values, 99)
extreme_low = np.where(X_min.values < low_th)
extreme_high = np.where(X_max.values > high_th)

In [17]:
extreme_low

(array([1833, 1847, 1849, 1850, 1858, 1861, 1864, 1866, 1873, 1874, 1875,
        1877, 1878, 1881, 1882, 1884, 1885, 1886, 1887, 1888, 1889, 1897]),)

In [6]:
def evaluation(X, X_hat):
    TS = X.shape[0] # num of test data
    M = X.shape[1] # num of senarios
    T = X.shape[2]
    # MAE; RMSE
    MAE = 0
    RMSE = 0
    
    for i in tqdm(range(TS)):
        for j in range(M):
            x = X[i, j, :]
            x_hat = X_hat[i, j, :]
            MAE = MAE + mean_absolute_error(x,x_hat)
            RMSE = RMSE + mean_squared_error(x, x_hat)
    MAE = MAE / (TS * M)
    RMSE = RMSE / (TS * M)
    print(f'MAE: {MAE:.4f}')
    print(f'RMSE: {RMSE:.4f}')

In [7]:
evaluation(X[extreme_low], X_hat_gan[extreme_low])
evaluation(X[extreme_low], X_hat_vae[extreme_low])
evaluation(X[extreme_low], X_hat_nf[extreme_low])
evaluation(X[extreme_low], X_hat_diff_base[extreme_low])
evaluation(X[extreme_low], X_hat_diff_phy[extreme_low])

100%|██████████| 22/22 [00:00<00:00, 119.76it/s]


MAE: 1.1140
RMSE: 2.4867


100%|██████████| 22/22 [00:00<00:00, 125.24it/s]


MAE: 1.1262
RMSE: 2.5243


100%|██████████| 22/22 [00:00<00:00, 123.47it/s]


MAE: 1.2851
RMSE: 3.7029


100%|██████████| 22/22 [00:00<00:00, 125.45it/s]


MAE: 1.1352
RMSE: 2.0371


100%|██████████| 22/22 [00:00<00:00, 124.08it/s]

MAE: 0.7988
RMSE: 1.0268


In [8]:
evaluation(X[extreme_high], X_hat_gan[extreme_high])
evaluation(X[extreme_high], X_hat_vae[extreme_high])
evaluation(X[extreme_high], X_hat_nf[extreme_high])
evaluation(X[extreme_high], X_hat_diff_base[extreme_high])
evaluation(X[extreme_high], X_hat_diff_phy[extreme_high])

100%|██████████| 22/22 [00:00<00:00, 120.95it/s]


MAE: 1.6429
RMSE: 5.0224


100%|██████████| 22/22 [00:00<00:00, 127.66it/s]


MAE: 1.6200
RMSE: 4.8953


100%|██████████| 22/22 [00:00<00:00, 127.87it/s]


MAE: 1.7573
RMSE: 5.8760


100%|██████████| 22/22 [00:00<00:00, 127.67it/s]


MAE: 1.5714
RMSE: 4.0944


100%|██████████| 22/22 [00:00<00:00, 127.43it/s]

MAE: 1.3179
RMSE: 3.0424
